In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def dual(Q, v, p):
    return v.T @ Q @ v + p.T @ v

def objective(Q, v, p, A, b, t):
    if np.any(b - A @ v <= 0):
        return float("NaN")
    return t*(v.T @ Q @ v + p.T @ v) - np.sum(np.log(b - A @ v))

def barrier_gradient(A, v, b):
    return A.T @ (1. /(b - A @ v))

def barrier_hessian(A, v, b):
    return A.T @ np.diag(1/(b - A @ v)**2) @ A

def gradient(Q, v, p, A, b, t):
    return t*(2 * Q @ v + p) + barrier_gradient(A, v, b)

def hessian(Q, v, p, A, b, t):
    return 2*t*Q + barrier_hessian(A, v, b)

In [9]:
def line_search(f, grad_f, v, dv, b, A, alpha=.5, beta=.9):
    step = 1

    while f(v + step * dv) > f(v) + alpha * step * grad_f(v).T @ dv and step > 1e-6:
        step *= beta
        
        if np.any(b - A @ (v + step * dv) <= 0):
            return step

    return step

def centering_step(Q, p, A, b, t, v0, eps, max_iter=500, alpha=.5, beta=.9):
    v_seq = [v0]
    v = v0
    i = 0

    while i < max_iter:
        try:
            dv = np.linalg.inv(hessian(Q, v, p, A, b, t)) @ gradient(Q, v, p, A, b, t)
        except np.linalg.LinAlgError:
            print("Hessian is not invertible")
            print(hessian(Q, v0, p, A, b, t))
            return None
        
        step = line_search(lambda v: objective(Q, v, p, A, b, t), lambda v: gradient(Q, v, p, A, b, t), v, dv, b, A, alpha, beta)
        v = v - step * dv
        v_seq.append(v)
        lambda_squared = gradient(Q, v, p, A, b, t).T @ dv
        if lambda_squared / 2 < eps:
            break
        i += 1

    return v_seq

In [10]:
def barr_method(Q, p, A, b, v0, eps, mu, alpha=.5, beta=.9, max_iter=500):
    m = A.shape[0]
    t = 1
    v_seq = [v0]
    v = v0
    f_obj = objective(Q, v, p, A, b, t)
    f_seq = [f_obj]
    i = 0
    while m/t >= eps:
        v_step = centering_step(Q, p, A, b, t, v, eps, max_iter=max_iter, alpha=alpha, beta=beta)
        v = v_step[-1]
        t = mu*t
        v_seq.append(v)
        f_obj = dual(Q, v, p)
        f_seq.append(f_obj)
        i += 1
        
    return v_seq, f_seq

def generate_parameters(n, d, reg=10):
    
    X = 3 * np.random.randn(n, d)
    y = 5 + 1.5* np.random.randn(n)

    Q = np.eye(n) / 2
    p = -y

    A = np.concatenate((X.T, - X.T), axis=0)
    b = reg * np.ones(2 * d)

    v0 = np.linalg.solve(A.T @ A, A.T @ b)

    return X, y, Q, p, A, b, v0

def generate_parameters_v0(n, d, reg=10):
    """See question 3."""
    
    X = 3 * np.random.randn(n, d)
    y = 5 + 1.5 * np.random.randn(n)

    Q = np.eye(n) / 2
    p = - y

    A = np.concatenate((X.T, - X.T), axis=0)
    b = reg * np.ones(2 * d)

    v0 = np.zeros(n)

    return X, y, Q, p, A, b, v0

Test of the method implemented 

In [11]:
n = 10
d = 5
reg = 10
X, y, Q, p, A, b, v0 = generate_parameters_v0(n, d, reg)

epsilon = 1e-9
alpha = 0.5
beta = 0.9
max_iter = 500

mu_list = [2, 15, 50, 100]
result = []
for mu in mu_list:
    v_seq, f_seq = barr_method(Q, y, A, b, v0, epsilon, mu, alpha, beta)
    print(f_seq)
    result.append([v_seq, f_seq])
f_opt = np.infty
for i in range(len(result)):
    f_opt = min(f_opt, result[i][1][-1])


for i, res in enumerate(result):
    plt.semilogy(res[1]-f_opt, label=f"mu={mu_list[i]}")
plt.xlabel("Number of iterations")
plt.ylabel("f(v) - f(v*)")
plt.legend()
plt.show()

Hessian is not invertible
[[ 1.28141408e+25  5.17135259e+24 -1.67373365e+25 -2.03414627e+25
   1.15635898e+25 -7.66703153e+24 -2.56068099e+24  2.13152269e+25
  -9.64648473e+24  9.48732318e+23]
 [ 5.17135259e+24  7.47379889e+24 -1.44313006e+25 -5.56326828e+24
   9.57990293e+24 -1.13782804e+25 -1.05203646e+25  2.54407546e+24
   1.01122716e+25  8.26947850e+24]
 [-1.67373365e+25 -1.44313006e+25  3.28015954e+25  2.27986603e+25
  -2.21056932e+25  2.18199777e+25  1.68644022e+25 -1.92079148e+25
  -7.35884848e+24 -1.24782879e+25]
 [-2.03414627e+25 -5.56326828e+24  2.27986603e+25  3.35900789e+25
  -1.59430656e+25  8.10188751e+24 -5.94850197e+23 -3.68118193e+25
   2.21920539e+25  2.36764269e+24]
 [ 1.15635898e+25  9.57990293e+24 -2.21056932e+25 -1.59430656e+25
   1.49163618e+25 -1.44746157e+25 -1.09636875e+25  1.37096113e+25
   4.06891199e+24  8.04938975e+24]
 [-7.66703153e+24 -1.13782804e+25  2.18199777e+25  8.10188751e+24
  -1.44746157e+25  1.73271438e+25  1.61216633e+25 -3.43710166e+24
  -1.57

TypeError: 'NoneType' object is not subscriptable